In [2]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

/home/spencersong/metamaterials_ai/src


In [3]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
work_folder = '../local_work'

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Inverse Model Only


In [4]:
from src.models import InverseModel


In [4]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-cnn-inconel-2022-07-05_04-49.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-cnn-inconel-2022-07-05_04-49.ckpt


In [5]:
model = InverseModel.load_from_checkpoint(filepath, strict=False)
model.eval()
x = torch.randn(1, 800)

with torch.no_grad():
    y_hat = model(x)
y_hat.shape

/home/parhammotameni/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/parhammotameni/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.mo

torch.Size([1, 14])

In [18]:
data_folder = '../local_data'
params = torch.load(
        Path(f"{data_folder}/stainless_steel.pt"))["laser_params"]
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
print(max_speed, max_spacing)
print(min_speed, min_spacing)

700.0 42.0
10.0 1.0


In [19]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:]) 
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = min_speed + (max_speed - min_speed)* y_hat[0][0] # TODO call the scale
    y_final[1] = min_spacing + (max_spacing - min_spacing) * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

In [20]:
# y_final2 = onehot_to_continuous2(y_hat)
print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

NameError: name 'y_hat' is not defined

# Direct Model Only


In [ ]:
from src.models import DirectModel

In [ ]:

filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_03-18.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

In [ ]:
model = DirectModel.load_from_checkpoint(filepath)
model.eval()
x = torch.randn(1, 14)

with torch.no_grad():
    y_hat = model(x)
y_hat.shape

In [ ]:
y_hat

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
